In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("Xtest.csv")

In [82]:
train_df = train_df.replace({np.nan: ''})
train_df.Password = train_df.Password.apply(str)
train_df.astype({'Password': 'U20', 'Times': 'i4'})

train_df = train_df.replace({np.nan: ''})
train_df.Password = train_df.Password.apply(str)
train_df.astype({'Password': 'U20'})

test_df = test_df.replace({np.nan: ''})
test_df.Password = test_df.Password.apply(str)
test_df.astype({'Password': 'U20'})

,Id,Password
0,0,ThaisCunha
1,1,697775113
2,2,922a16922a
3,3,andy74
4,4,joemack
...,...,...
1037870,1037870,poo333
1037871,1037871,lpruss
1037872,1037872,tiggerbel
1037873,1037873,MINOFXJF


In [83]:
def count_digits_ratio(x):
    if len(x) == 0:
        return 0
    digits = 0
    for i in range(len(x)):
        if x[i].isdigit():
            digits += 1
    return min(digits / len(x), 1 - digits / len(x))

def count_special_chars_ratio(x):
    if len(x) == 0:
        return 0
    special_chars = 0
    for i in range(len(x)):
        if not x[i].isalnum():
            special_chars += 1
    return min(special_chars / len(x), 1 - special_chars / len(x))

def count_uppercase_ratio(x):
    if len(x) == 0:
        return 0
    uppercase = 0
    for i in range(len(x)):
        if x[i].isupper():
            uppercase += 1
    return min(uppercase / len(x), 1 - uppercase / len(x))

train_df["Length"] = train_df["Password"].apply(lambda x: len(x))
train_df["Unique Length"] = train_df["Password"].apply(lambda x: len(set(x)))
train_df["Digits ratio"] = train_df["Password"].apply(count_digits_ratio)
train_df["Special Characters ratio"] = train_df["Password"].apply(count_special_chars_ratio)
train_df["Uppercase ratio"] = train_df["Password"].apply(count_uppercase_ratio)

test_df["Length"] = test_df["Password"].apply(lambda x: len(x))
test_df["Unique Length"] = test_df["Password"].apply(lambda x: len(set(x)))
test_df["Digits Ratio"] = test_df["Password"].apply(count_digits_ratio)
test_df["Special Characters Ratio"] = test_df["Password"].apply(count_special_chars_ratio)
test_df["Uppercase Ratio"] = test_df["Password"].apply(count_uppercase_ratio)


train_df

,Password,Times,Length,Unique Length,Digits ratio,Special Characters ratio,Uppercase ratio,Entropy
0,631XniVx2lS5I,2,13,13,0.384615,0.0,0.307692,7
1,LEGIT747,1,8,7,0.375000,0.0,0.375000,4
2,742364es,1,8,7,0.250000,0.0,0.000000,10
3,3846696477,1,10,6,0.000000,0.0,0.000000,0
4,laurahop,2,8,7,0.000000,0.0,0.000000,0
...,...,...,...,...,...,...,...,...
4151491,7503ri,1,6,6,0.333333,0.0,0.000000,6
4151492,trou,2,4,4,0.000000,0.0,0.000000,0
4151493,300762,7,6,5,0.000000,0.0,0.000000,0
4151494,seaurchin007,1,12,11,0.250000,0.0,0.000000,4


In [94]:
from sklearn.model_selection import train_test_split

x = pd.DataFrame(train_df, columns = ["Length", "Unique Length", "Digits Ratio", "Special Characters Ratio", "Uppercase Ratio"])
y = pd.DataFrame(train_df, columns = ["Times"])

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.1,
                                                    random_state=42)

In [95]:
from sklearn import tree
model = tree.DecisionTreeRegressor()
model.fit(x_train, y_train)

DecisionTreeRegressor()

In [96]:
y_pred = model.predict(x_test)

y_pred

array([1.59139802, 1.10691358, 1.28600797, ..., 1.48369979, 1.48369979,
       5.19973005])

In [97]:
y_test

,Times
2808555,1
763838,1
2715631,1
747473,1
603659,1
...,...
1085253,1
15451,1
2270760,1
1059647,1


In [98]:
terms_to_sum = 0
assert len(y_test) == len(y_pred)
for i in range(len(y_pred) - 1):
    terms_to_sum += (math.log(y_pred[i] + 1) - math.log(y_test.iloc[0]["Times"] + 1)) ** 2.0
print((terms_to_sum * (1.0/len(y_test))) ** 0.5)

0.42496004933532083
